<a href="https://colab.research.google.com/github/jcity/dotfiles/blob/master/Copy_of_cheap_strike_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pandas_datareader
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=24eb288564457e0ede0d8e663da80bec74b1305e23e75bdf98cf8d64f6e13629
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [0]:
#Imports, keys here are pandas_datareader allows us to download easily and
#yfinance allows us to get into yahoo
import pandas_datareader.data as data
import yfinance as yf
import pandas as pd

yf.pdr_override()

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [0]:
def cheap_opts(symbol, calls_or_puts):
  '''
  This searches all possible expiry dates and finds contracts.
  '''
  import sys, os
  from datetime import date

  #We're going to suppress prints b/c datareader is annoying then restore printing so this helps
  #old_stdout = sys.stdout
  #sys.stdout = open(os.devnull, "w")

  finaldf = pd.DataFrame()
  ticker = yf.Ticker(symbol)
  stock_price = data.get_data_yahoo(symbol, end_date = date.today())['Close'][-1]
  for opt_date in ticker.options:
    opt = ticker.option_chain(opt_date)
    if calls_or_puts == 'puts':
      opt.puts.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.puts)
    else:
      opt.calls.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.calls)
  return finaldf, stock_price

In [0]:
%%time

calls_or_puts = 'puts'
symbol = 'acb'

results, stock_price = cheap_opts(symbol, calls_or_puts)

[*********************100%***********************]  1 of 1 completed
CPU times: user 479 ms, sys: 39.5 ms, total: 518 ms
Wall time: 1.84 s


In [0]:
returned = results
# Do some final formatting changes
returned = returned.drop(columns = ['contractSymbol', 'lastTradeDate', 'contractSize', 'currency'])
if calls_or_puts == 'puts':
    returned.insert(3, 'dist OTM', stock_price - returned['strike'])
else:
    returned.insert(3, 'dist OTM', returned['strike'] - stock_price)
  
returned.insert(4, '% OTM', returned['dist OTM']/stock_price*100)
returned['value'] = returned['openInterest']*returned['lastPrice']*100

# Do some filtering
returned = returned[returned['% OTM'] < 50]
returned = returned[returned['lastPrice'] < 15]
returned = returned[returned.openInterest > 50]
returned = returned[returned['opt_date'] > '2020-05-24']
#returned = returned[returned['percentChange'] < -0]

# Sort and display
returned.sort_values(inplace=True, by='lastPrice', ascending=True)
returned = returned.head(40)
returned.reset_index(drop=True, inplace=True)
final = returned.style.set_precision(3).set_caption('<H1>' + symbol + ' ' + calls_or_puts.upper() + \
                                          '   (Current = $' + str(round(stock_price, 2)) + ')')
final

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
0,2022-01-21,12.000,11.430,5.400,31.034,11.150,11.600,-0.170,-1.466,1.000,77,3.127,False,88011.000


In [0]:
# Download the CSV and you're done!
from google.colab import files
filename = 'cheap_' + symbol + '_' + calls_or_puts + '.csv'

returned.to_csv(filename)
# returned.to_csv('unusual_calls_activity_5_15_exp.csv')
files.download(filename)

In [0]:
%%time
top = returned.sort_values(by='lastPrice', ascending=True) #.head(70)
top.sort_values(by='% OTM', inplace=True, ascending=True)
a = top.style.set_caption(calls_or_puts.upper())
display(a)
# top[top['inTheMoney']== True]

In [0]:
#top.to_csv('filtered_puts_05_15.csv')
top.to_csv('filtered_'+calls_or_puts+'_05_15.csv')

In [0]:
otm=returned['inTheMoney'] == False
#returned.loc[otm,'Distance OTM']=999-returned.lastPrice
returned.style.set_precision(2)

In [0]:
#top[top['inTheMoney']== True]